# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
 COMP_CLOSENESS_DICT = {'Subj 1 blocking Subj 2': "competitive",
'Subj 2 Only': "no_comp",
'Subj 2 blocking Subj 1': "competitive",
'Subj 1 then Subj 2': "competitive", 
'Subj 1 Only': "no_comp",
'Subj 2 then Subj 1': "competitive",
'Close Call': "competitive",
'After trial': "no_comp"}

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [16]:
TRIAL_LABELS_DF = pd.read_excel("./proc/rce2_finalized_trial_labels.xlsx")

In [17]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_05_lfp_spectral_sleap_spikes.pkl")

In [18]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)

In [19]:
TRIAL_LABELS_DF["current_subject"] = TRIAL_LABELS_DF["current_subject"].astype(str)
TRIAL_LABELS_DF["competition_closeness"] = TRIAL_LABELS_DF["competition_closeness"].map(COMP_CLOSENESS_DICT)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

In [73]:
import numpy as np

def mask_slices(array_2d, slice_index):
    """
    Masks elements outside a specified slice in a 2D array with NaN.

    Parameters:
    - array_2d (np.ndarray): The input 2D array to mask.
    - slice_index (tuple): A tuple of two integers that specifies the start and end indices 
                           of the slice to retain. The elements outside this slice are set to NaN.

    Returns:
    - np.ndarray: A 2D array with elements outside the specified slice set to NaN.

    Raises:
    - ValueError: If `slice_index` is not a tuple or does not contain exactly two elements.
    - IndexError: If the slice indices are out of the array bounds.
    """
    if not isinstance(slice_index, tuple) or len(slice_index) != 2:
        raise ValueError("slice_index must be a tuple of two integers (start, end).")

    try:
        result = array_2d.copy()
        mask_2d = np.ones(result.shape, dtype=bool)
        mask_2d[slice_index[0]:slice_index[1]] = False

        # Set values outside the defined slice to NaN
        result[mask_2d] = np.nan
        return result

    except IndexError as e:
        raise IndexError(f"Slice index out of bounds: {e}")
    except Exception as e:
        raise RuntimeError(f"An error occurred while masking the array: {e}")


In [89]:
def overlay_arrays(array1, array2):
    """
    Overlays two 2D NumPy arrays of the same shape, preferring non-NaN values from the first array.
    If both arrays have a non-NaN value at the same position, the value from the first array is used.

    Parameters:
    - array1 (np.ndarray): The primary 2D array.
    - array2 (np.ndarray): The secondary 2D array, used only where array1 has NaNs.

    Returns:
    - np.ndarray: A 2D array composed of the overlaid results of array1 and array2.

    Raises:
    - ValueError: If `array1` and `array2` do not have the same shape.
    """
    if array1.shape != array2.shape:
        raise ValueError("Both arrays must have the same shape.")

    # Create a copy of the first array to ensure that no changes are made to the original
    result = np.copy(array1)

    # Find indices where array1 is NaN and array2 is not NaN
    mask = np.isnan(array1) & ~np.isnan(array2)

    # Place non-NaN values from array2 where array1 has NaNs
    result[mask] = array2[mask]

    return result

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [20]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [21]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject',
       ...
       'agent_locations', 'subject_thorax_velocity', 'agent_thorax_velocity',
       'subject_thorax_to_reward_port', 'agent_thorax_to_reward_port',
       'recording_name', 'spike_clusters', 'spike_times', 'neuron_average_fr',
       'neuron_average_timestamps'],
      dtype='object', length=173)

In [22]:
TRIAL_LABELS_DF.columns

Index(['tracked_subject', 'box_number', 'sleap_name', 'video_name',
       'current_subject', 'tone_start_frame', 'reward_start_frame',
       'tone_stop_frame', 'condition', 'competition_closeness', 'notes',
       'experiment', 'session_dir', 'all_subjects', 'tone_start_timestamp',
       'tone_stop_timestamp', 'trial_label'],
      dtype='object')

# Editing the dataframes

In [23]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,agent_locations,subject_thorax_velocity,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps
0,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,NaN,"[14.84, 13.97, 13.13, 12.32, 11.54, 10.78, 10....",NaN,"[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
1,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,1.1,...,"[[[40.039741738777444, 9.47029992556593], [42....","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
2,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[1029, 1229], [3425, 3624], [5721, 5920], [75...","[[102, 105], [110, 148], [193, 199], [200, 202...","[[24984, 31321], [39533, 39547], [39554, 39559...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,"[[[57.86163468579171, 31.554562356382696], [59...","[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550...","[[1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
3,2,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,1.2,...,NaN,"[5.184, 4.777, 4.383, 4.0, 3.629, 3.271, 2.926...",NaN,"[6.223, 6.227, 6.234, 6.24, 6.203, 6.2, 6.203,...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550...","[[1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
4,2,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,1.1,...,"[[[31.3

In [24]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.1',
       '20230617_115521_standard_comp_to_omission_D1_subj_1-1_and_1-2.3',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.1',
       '20230618_100636_standard_comp_to_omission_D2_subj_1-4_and_1-1.2',
       '20230619_115321_standard_comp_to_omission_D3_subj_1-2_and_1-4.4',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.1',
       '20230620_114347_standard_comp_to_omission_D4_subj_1-2_and_1-1.2',
       '20230621_111240_standard_comp_to_omission_D5_subj_1-4_and_1-2.1',
       '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_and_1-2.1',
       '20230624_105855_standard_comp_to_both_rewarded_D3_subj_1-2_and_1-4.1',
       '20230625_112913_standard_comp_to_both_rewarded_D4_subj_1-1_and_1-4.1',
       '20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs2-2and1-2vs2-1.1',
       '20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs2-2and1-2vs2-1.4',
 

In [25]:
LFP_SPECTRAL_DF["tracked_subject"]

0          [1.1]
1     [1.1, 1.2]
2     [1.1, 1.2]
3          [1.2]
4     [1.1, 1.4]
5          [1.1]
6          [1.4]
7     [1.1, 1.4]
8          [1.4]
9     [1.1, 1.2]
10         [1.1]
11         [1.2]
12    [1.1, 1.2]
13         [1.4]
14    [1.2, 1.4]
15    [1.1, 1.2]
16    [1.1, 1.2]
17    [1.2, 1.4]
18    [1.1, 1.4]
19    [1.1, 1.4]
20    [1.1, 1.2]
21    [1.1, 2.2]
22    [1.1, 2.2]
23    [1.1, 1.2]
24    [1.1, 1.4]
25    [1.1, 2.1]
Name: tracked_subject, dtype: object

In [26]:
LFP_SPECTRAL_DF["tracked_subject"] = LFP_SPECTRAL_DF["tracked_subject"].apply(lambda x: "_".join([str(subj) for subj in x]))

In [41]:
LFP_SPECTRAL_DF["current_tracked_subject"] = LFP_SPECTRAL_DF["tracked_subject"]

In [42]:
# Example with multiple columns being aggregated
GROUPED_TRIAL_LABELS_DF = TRIAL_LABELS_DF.groupby(['video_name', "current_subject"]).agg({col: list for col in [_ for _ in TRIAL_LABELS_DF.columns if _ not in ["video_name", 'current_subject']]  # Aggregating into a list
    # Other columns can be aggregated differently here
}).reset_index()

In [43]:
GROUPED_TRIAL_LABELS_DF["session_dir"] = GROUPED_TRIAL_LABELS_DF["session_dir"].apply(lambda x: x[0])


In [44]:
GROUPED_TRIAL_LABELS_DF["sleap_name"] = GROUPED_TRIAL_LABELS_DF["sleap_name"].apply(lambda x: x[0])

In [45]:
GROUPED_TRIAL_LABELS_DF["trial_notes"] = GROUPED_TRIAL_LABELS_DF["notes"]

In [46]:
GROUPED_TRIAL_LABELS_DF.head()

,video_name,current_subject,tracked_subject,box_number,sleap_name,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,notes,experiment,session_dir,all_subjects,tone_start_timestamp,tone_stop_timestamp,trial_label,trial_notes
0,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230612_112630_standard_comp_to_training_D1_s...,"[1125, 3519, 5815, 7612, 8709, 9708, 11604, 13...","[1225, 3619, 5915, 7712, 8809, 9808, 11704, 13...","[1324, 3720, 6014, 7811, 8910, 9907, 11804, 13...","[1.1, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, ...","[competitive, no_comp, competitive, no_comp, n...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[standard, standard, standard, standard, stand...",20230612_112630_standard_comp_to_training_D1_s...,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[1126742, 3526740, 5826740, 7626736, 8726734, ...","[1326741, 3726740, 6026737, 7826735, 8926734, ...","[win, lose, lose, lose, lose, lose, lose, lose...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,20230612_112630_standard_comp_to_training_D1_s...,1.2,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230612_112630_standard_comp_to_training_D1_s...,"[1125, 3519, 5815, 7612, 8709, 9708, 11604, 13...","[1225, 3619, 5915, 7712, 8809, 9808, 11704, 13...","[1324, 3720, 6014, 7811, 8910, 9907, 11804, 13...","[1.1, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, ...","[competitive, no_comp, competitive, no_comp, n...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[standard, standard, standard, standard, stand...",20230612_112630_standard_comp_to_training_D1_s...,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[1126742, 3526740, 5826740, 7626736, 8726734, ...","[1326741, 3726740, 6026737, 7826735, 8926734, ...","[lose, win, win, win, win, win, win, win, win,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,20230612_112630_standard_comp_to_training_D1_s...,1.1,"[['1.1'], ['1.1'], ['1.1'], ['1.1'], ['1.1'], ...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",20230612_112630_standard_comp_to_training_D1_s...,"[37056, 38254, 39352, 40650, 41747, 42746, 437...","[37156, 38354, 39452, 40750, 41847, 42846, 438...","[37256, 38453, 39552, 40849, 41947, 42945, 439...","[rewarded, rewarded, rewarded, rewarded, rewar...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[standard, standard, standard, standard, stand...",20230612_112630_standard_comp_to_training_D1_s...,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[37126704, 38326703, 39426702, 40726701, 41826...","[37326704, 38526703, 39626701, 40926700, 42026...","[rewarded, rewarded, rewarded, rewarded, rewar...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,20230613_105657_standard_comp_to_training_D2_s...,1.1,"[['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230613_105657_standard_comp_to_training_D2_s...,"[916, 3312, 5608, 7405, 8502, 9501, 11397, 128...","[1016, 3412, 5708, 7505, 8602, 9601, 11497, 12...","[1117, 3513, 5808, 7604, 8703, 9700, 11596, 13...","[1.4, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[no_comp, no_comp, competitive, competitive, c...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[standard, standard, standard, standard, stand...",20230613_105657_standard_comp_to_training_D2_s...,"[['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...","[918755, 3318755, 5618754, 7418755, 8518752, 9...","[1118758, 3518757, 5818753, 7618752, 8718757, ...","[lose, win, win, win, win, win, win, win, win,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
4,20230613_105657_standard_comp_to_training_D2_s...,1.4,"[['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230613_105657_standard_comp_to_training_D2_s.

## Merging the labels and LFP spectral

In [55]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

video_name
current_subject
tracked_subject
box_number
sleap_name
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
trial_notes
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band
mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_RMS_filtered_power_all_frequencies_all_windows
MD_RMS_filtered_power_all_frequencies_all_windows
LH_RMS_filtered_power_all_frequencies_all_windows
BLA_RMS_filtered_power_all_frequencies_all_window

In [47]:
TRIALS_AND_SPECTRAL_DF = pd.merge(GROUPED_TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["current_subject", "video_name"], how="inner", suffixes=('', '_y'))

In [48]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "_y" in col], errors="ignore")

In [56]:
TRIALS_AND_SPECTRAL_DF["start_frame"]

0     41000
1         1
2         1
3         1
4     32792
5         1
6     32792
7      2027
8     32860
9      2027
10    32860
11    35000
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19    37500
20        1
21        1
22    38000
Name: start_frame, dtype: int64

In [58]:
TRIALS_AND_SPECTRAL_DF["start_stop_frame"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: (x["start_frame"], x["stop_frame"]), axis=1)


In [77]:
TRIALS_AND_SPECTRAL_DF["start_stop_frame"] 

0     (41000, 79050)
1         (1, 39500)
2         (1, 39500)
3         (1, 32316)
4     (32792, 68494)
5         (1, 32316)
6     (32792, 68494)
7      (2027, 32239)
8     (32860, 68287)
9      (2027, 32239)
10    (32860, 68287)
11    (35000, 69766)
12        (1, 33500)
13        (1, 73111)
14        (1, 73111)
15        (1, 72635)
16        (1, 73292)
17        (1, 73292)
18        (1, 31500)
19    (37500, 69199)
20        (1, 31500)
21        (1, 31500)
22    (38000, 69884)
Name: start_stop_frame, dtype: object

- Making all parts of the numpy array that is not part of slice Nans

In [113]:
sleap_columns = ["subject_locations", 
                 "agent_locations", 
                 "subject_thorax_velocity", 
                 "agent_thorax_velocity", 
                 "subject_thorax_to_reward_port", 
                 "agent_thorax_to_reward_port"]

In [ ]:
for col in sleap_columns:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: mask_slices(x[col], x["start_stop_frame"]), axis=1)


In [ ]:
raise ValueError()

- Merging rows

In [94]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

video_name
current_subject
tracked_subject
box_number
sleap_name
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
trial_notes
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band
mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_RMS_filtered_power_all_frequencies_all_windows
MD_RMS_filtered_power_all_frequencies_all_windows
LH_RMS_filtered_power_all_frequencies_all_windows
BLA_RMS_filtered_power_all_frequencies_all_window

In [98]:
aggregation_dict = {col:"first" for col in TRIALS_AND_SPECTRAL_DF if col not in ['subject_locations', "current_subject", "session_dir"]}

In [99]:
aggregation_dict["subject_locations"] = list

In [101]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.groupby(["current_subject", "session_dir"]).agg(aggregation_dict).reset_index()

In [106]:
TRIALS_AND_SPECTRAL_DF[drop_columns].head()

video_name  \
current_subject session_dir                                                                                             
1.1             20230617_115521_standard_comp_to_omission_D1_su...  20230617_115521_standard_comp_to_omission_D1_s...   
                20230618_100636_standard_comp_to_omission_D2_su...  20230618_100636_standard_comp_to_omission_D2_s...   
                20230620_114347_standard_comp_to_omission_D4_su...  20230620_114347_standard_comp_to_omission_D4_s...   
                20230622_110832_standard_comp_to_both_rewarded_...  20230622_110832_standard_comp_to_both_rewarded...   
                20230625_112913_standard_comp_to_both_rewarded_...  20230625_112913_standard_comp_to_both_rewarded...   

                                                                                                      tracked_subject  \
current_subject session_dir                                                                                             
1.1             20230617_115521_standard_comp_to_omission_D1_su...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230618_100636_standard_comp_to_omission_D2_su...  [['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...   
                20230620_114347_standard_comp_to_omission_D4_su...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230622_110832_standard_comp_to_both_rewarded_...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230625_112913_standard_comp_to_both_rewarded_...  [['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...   

                                                                                                           box_number  \
current_subject session_dir                                                                                             
1.1             20230617_115521_standard_comp_to_omission_D1_su...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
                20230618_100636_standard_comp_to_omission_D2_su...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
                20230620_114347_standard_comp_to_omission_D4_su...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
                20230622_110832_standard_comp_to_both_rewarded_...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
                20230625_112913_standard_comp_to_both_rewarded_...  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                                                                                           sleap_name  \
current_subject session_dir                                                                                             
1.1             20230617_115521_standard_comp_to_omission_D1_su...  20230617_115521_standard_comp_to_omission_D1_s...   
                20230618_100636_standard_comp_to_omission_D2_su...  20230618_100636_standard_comp_to_omission_D2_s...   
                20230620_114347_standard_comp_to_omission_D4_su...  20230620_114347_standard_comp_to_omission_D4_s...   
                20230622_110832_standard_comp_to_both_rewarded_...  20230622_110832_standard_comp_to_both_rewarded...   
                20230625_112913_standard_comp_to_both_rewarded_...  20230625_112913_standard_comp_to_both_rewarded...   

                                                                                                                notes  \
current_subject session_dir                                                                                             
1.1             20230617_115521_standard_comp_to_omission_D1_su...  [nan, nan, nan, nan, nan, nan, nan, nan, 1.2 g...   
                20230618_100636_standard_comp_to_omission_D2_su...  [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...   
                20230620_114347_standard_comp_to_omission_D4_su...  [nan, nan, 1.1 still at port at 8791, nan, nan...   
                20230622_110832_standard_comp_to_both_rewarded_...  [nan, nan, nan, nan, nan, nan, nan, nan, n

In [104]:
drop_columns = ["video_name", "tracked_subject", "box_number", "sleap_name", "notes", "start_frame", "stop_frame", "in_video_subjects", "current_tracked_subject", "start_stop_frame"]

In [ ]:

body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent

recording_name

In [ ]:
agent_locations
subject_thorax_velocity
agent_thorax_velocity
subject_thorax_to_reward_port
agent_thorax_to_reward_port

In [103]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

video_name
tracked_subject
box_number
sleap_name
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
trial_notes
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band
mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_RMS_filtered_power_all_frequencies_all_windows
MD_RMS_filtered_power_all_frequencies_all_windows
LH_RMS_filtered_power_all_frequencies_all_windows
BLA_RMS_filtered_power_all_frequencies_all_windows
vHPC_RMS_filtered_power_al

In [ ]:
raise ValueError()

In [102]:
TRIALS_AND_SPECTRAL_DF

video_name  \
current_subject session_dir                                                                                             
1.1             20230617_115521_standard_comp_to_omission_D1_su...  20230617_115521_standard_comp_to_omission_D1_s...   
                20230618_100636_standard_comp_to_omission_D2_su...  20230618_100636_standard_comp_to_omission_D2_s...   
                20230620_114347_standard_comp_to_omission_D4_su...  20230620_114347_standard_comp_to_omission_D4_s...   
                20230622_110832_standard_comp_to_both_rewarded_...  20230622_110832_standard_comp_to_both_rewarded...   
                20230625_112913_standard_comp_to_both_rewarded_...  20230625_112913_standard_comp_to_both_rewarded...   
                20230628_111202_standard_comp_to_novel_agent_D1...  20230628_111202_standard_comp_to_novel_agent_D...   
                20230629_111937_standard_comp_to_novel_agent_D2...  20230629_111937_standard_comp_to_novel_agent_D...   
1.2             20230617_115521_standard_comp_to_omission_D1_su...  20230617_115521_standard_comp_to_omission_D1_s...   
                20230620_114347_standard_comp_to_omission_D4_su...  20230620_114347_standard_comp_to_omission_D4_s...   
                20230622_110832_standard_comp_to_both_rewarded_...  20230622_110832_standard_comp_to_both_rewarded...   
                20230628_111202_standard_comp_to_novel_agent_D1...  20230628_111202_standard_comp_to_novel_agent_D...   
1.4             20230618_100636_standard_comp_to_omission_D2_su...  20230618_100636_standard_comp_to_omission_D2_s...   
                20230621_111240_standard_comp_to_omission_D5_su...  20230621_111240_standard_comp_to_omission_D5_s...   
                20230624_105855_standard_comp_to_both_rewarded_...  20230624_105855_standard_comp_to_both_rewarded...   
                20230625_112913_standard_comp_to_both_rewarded_...  20230625_112913_standard_comp_to_both_rewarded...   

                                                                                                      tracked_subject  \
current_subject session_dir                                                                                             
1.1             20230617_115521_standard_comp_to_omission_D1_su...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230618_100636_standard_comp_to_omission_D2_su...  [['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...   
                20230620_114347_standard_comp_to_omission_D4_su...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230622_110832_standard_comp_to_both_rewarded_...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230625_112913_standard_comp_to_both_rewarded_...  [['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...   
                20230628_111202_standard_comp_to_novel_agent_D1...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230629_111937_standard_comp_to_novel_agent_D2...  [['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...   
1.2             20230617_115521_standard_comp_to_omission_D1_su...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230620_114347_standard_comp_to_omission_D4_su...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230622_110832_standard_comp_to_both_rewarded_...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
                20230628_111202_standard_comp_to_novel_agent_D1...  [['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...   
1.4             20230618_100636_standard_comp_to_omission_D2_su...  [['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...   
                20230621_111240_standard_comp_to_omission_D5_su...  [['1.2', '1.4'], ['1.2', '1.4'], ['1.2', '1.4'...   
                20230624_105855_standard_comp_to_both_rewarded_...  [['1.2', '1.4'], ['1.2', '1.4'], ['1.2', '1.4'...   
                20230625_112913_standard_comp_to_both_rewarded_...  [['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4

In [91]:
def combine_grouped_rows(df, array_columns):
    """
    Combine rows within groups of a DataFrame. Uses the `overlay_arrays` for specified columns
    and takes the first instance for other columns.

    Parameters:
    - df (pd.DataFrame): DataFrame to process.
    - array_columns (list): List of column names in `df` that contain array values to be combined using `overlay_arrays`.

    Returns:
    - pd.DataFrame: DataFrame after combining rows within groups.
    """
    def custom_aggregator(x):
        if x.name in array_columns:
            # Reduce the column by overlaying arrays
            return x.dropna().aggregate(lambda arrays: arrays.reduce(overlay_arrays))
        else:
            # For other columns, simply return the first element
            return x.iloc[0]

    # Apply the custom aggregator to each column individually
    return df.groupby(df.index).aggregate(custom_aggregator)


In [93]:
combine_grouped_rows(TRIALS_AND_SPECTRAL_DF, ["subject_locations"])

AttributeError: 'Series' object has no attribute 'reduce'

In [90]:
TRIALS_AND_SPECTRAL_DF.head()

,video_name,current_subject,tracked_subject,box_number,sleap_name,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,agent_thorax_velocity,subject_thorax_to_reward_port,agent_thorax_to_reward_port,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps,current_tracked_subject,start_stop_frame
0,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230617_115521_standard_comp_to_omission_D1_s...,"[1029, 3425, 5721, 7517, 8615, 9613, 11510, 13...","[1129, 3525, 5821, 7617, 8715, 9713, 11610, 13...","[1229, 3624, 5920, 7717, 8815, 9812, 11709, 13...","[1.2, 1.1, 1.1, 1.2, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[competitive, competitive, competitive, no_com...",...,NaN,"[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",NaN,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",1.1,"(41000, 79050)"
1,20230617_115521_standard_comp_to_omission_D1_s...,1.1,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230617_115521_standard_comp_to_omission_D1_s...,"[1029, 3425, 5721, 7517, 8615, 9613, 11510, 13...","[1129, 3525, 5821, 7617, 8715, 9713, 11610, 13...","[1229, 3624, 5920, 7717, 8815, 9812, 11709, 13...","[1.2, 1.1, 1.1, 1.2, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[competitive, competitive, competitive, no_com...",...,"[11.19, 10.55, 9.92, 9.31, 8.734, 8.18, 7.64, ...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223.0, 2367433.0, 2569212.0, 3969647.0, 3...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",1.1_1.2,"(1, 39500)"
2,20230617_115521_standard_comp_to_omission_D1_s...,1.2,"[['1.1', '1.2'], ['1.1', '1.2'], ['1.1', '1.2'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230617_115521_standard_comp_to_omission_D1_s...,"[1029, 3425, 5721, 7517, 8615, 9613, 11510, 13...","[1129, 3525, 5821, 7617, 8715, 9713, 11610, 13...","[1229, 3624, 5920, 7717, 8815, 9812, 11709, 13...","[1.2, 1.1, 1.1, 1.2, 1.1, 1.1, 1.1, 1.1, 1.1, ...","[competitive, competitive, competitive, no_com...",...,"[8.96, 8.195, 7.54, 7.004, 6.605, 6.348, 6.227...","[25.6, 25.84, 25.86, 25.86, 25.86, 25.86, 25.8...","[11.19, 11.234, 11.73, 12.34, 12.56, 13.58, 13...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958.0, 5233.0, 6331.0, 7475.0, 8599.0, 10550...","[[1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",1.1_1.2,"(1, 39500)"
3,20230618_100636_standard_comp_to_omission_D2_s...,1.1,"[['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '1.4'...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",20230618_100636_standard_comp_to_omission_D2_s...,"[1098, 3492, 5788, 7585, 8684, 9681, 11577, 13...","[1198, 3592, 5888, 7685, 8784, 9781, 11677, 13...","[1297, 3693, 5988, 7784, 8883, 9880, 11777, 13...","[1.4, 1.1, 1.1, 1.4, 1.1, 1.4, 1.4, 1.4, 1.1, ...","[no_comp, no_comp, no_comp, no_comp, no_comp, ...",...,"[0.6333, 0.5635, 0.4963, 0.4321, 0.3713, 0.313...","[5.727, 5.91, 5.93, 5.965, 5.99, 5.992, 6.355,...","[13.75, 14.18, 13.734, 13.74, 13.76, 13.84, 13...",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506.0, 1419.0, 2358.0, 5104.0, 6785.0, 7620....","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400...",1.1_1.4,"(1, 32316)"
4,20230618_100636_standard_comp_to_omission_D2_s...,1.4,"[['1.1', '1.4'], ['1.1', '1.4'], ['1.1', '

In [88]:
overlay_arrays(TRIALS_AND_SPECTRAL_DF["subject_locations"].iloc[0], TRIALS_AND_SPECTRAL_DF["subject_locations"].iloc[1])

array([[[        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan]],

       [[58.36080529, 31.53608315],
        [60.50075747, 33.33426566],
        [58.86053103, 30.0147542 ],
        [51.65891389, 32.95596033],
        [56.1104419 , 33.35010221],
        [59.55146672, 31.86303096]],

       [[58.3191749 , 31.17241468],
        [60.53709931, 32.9257053 ],
        [58.86053103, 30.0147542 ],
        [52.16234754, 32.987791  ],
        [56.60686229, 33.30918813],
        [59.45771358, 30.50920622]],

       ...,

       [[46.63059899, 35.18794659],
        [43.87864954, 34.77844084],
        [45.02844335, 37.27501554],
        [49.4834674 , 30.09450087],
        [47.21144128, 32.29533647],
        [44.98410935, 36.20014797]],

       [[46.64254061, 35.18381952],
        [43.89105306, 34.77964883],
        [45.01492533, 37.28756813],
       

In [87]:
TRIALS_AND_SPECTRAL_DF["subject_locations"].iloc[0].shape

(79051, 6, 2)

In [72]:
example_arr.shape

(79051, 6, 2)

In [74]:
mask_slices(example_arr, (41000, 79050))

array([[[        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan]],

       [[        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan]],

       [[        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan],
        [        nan,         nan]],

       ...,

       [[46.63059899, 35.18794659],
        [43.87864954, 34.77844084],
        [45.02844335, 37.27501554],
        [49.4834674 , 30.09450087],
        [47.21144128, 32.29533647],
        [44.98410935, 36.20014797]],

       [[46.64254061, 35.18381952],
        [43.89105306, 34.77964883],
        [45.01492533, 37.28756813],
       

In [ ]:
# Create a 2D array
array_2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Slice definition for rows and columns
row_start, row_end = 1, 3
col_start, col_end = 0, 2

# Create mask for rows and columns
mask_2d = np.ones(array_2d.shape, dtype=bool)
mask_2d[row_start:row_end, col_start:col_end] = False

# Set values outside the defined slice to NaN
array_2d[mask_2d] = np.nan

print(array_2d)

In [37]:
raise ValueError()

ValueError: 

In [ ]:
for index, row in TRIALS_AND_SPECTRAL_DF.iterrows():
    print(row["video_name"])

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
raise ValueError()

In [ ]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["current_subject", "video_name"], how="inner", suffixes=('', '_y'))

In [ ]:
# TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "_y" in col], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
TRIALS_AND_SPECTRAL_DF.columns

In [ ]:
TRIALS_AND_SPECTRAL_DF

# Calculating the relevant timestamps

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 30 * 20000

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 20 * 20000

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

# Filtering out the SLEAP posed for during trials

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head()

In [ ]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [ ]:
sleap_columns

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_timestamps"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["video_frame"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["subject_thorax_to_reward_port"].iloc[0].shape

In [ ]:
for col in sorted(sleap_columns):
    updated_item_col = "baseline_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "baseline_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in sorted(sleap_columns):
    updated_item_col = "trial_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "trial_video_timestamps".format(col)
    if "agent" in col:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
    else:
        TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_subject_thorax_velocity"].apply(lambda x: np.isnan(x).any())]["baseline_subject_locations"]

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

- Filtering coherence

In [ ]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
coherence_columns

In [ ]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [ ]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
granger_columns

In [ ]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [ ]:
TRIALS_AND_SPECTRAL_DF.head()

In [ ]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
power_columns

In [ ]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_power_timestamps"]

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

# Filtering out phase

In [ ]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [ ]:
phase_columns = phase_columns + band_columns +lfp_columns

In [ ]:
phase_columns

In [ ]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

In [ ]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_lfp_timestamps"]

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

# Filtering out spikes

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape

# Filtering for rows that are in the video

In [ ]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [ ]:
TRIALS_AND_SPECTRAL_DF

In [ ]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()